In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install langchain transformers faiss-cpu sentence-transformers
!pip install langchain_community
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [6]:
    df = pd.read_csv('/kaggle/input/stackoverflow-python/qa_stackoverflow_python_high_score_1.csv')


In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd
from langchain.schema import Document


# Load and process the PDF
def build_and_save_faiss_store(store_path="faiss_store"):
    # Load the PDF into LangChain documents
    df = pd.read_csv('/kaggle/input/stackoverflow-python/qa_stackoverflow_python_high_score_1.csv')

    # Create FAISS vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(df['Answer'].tolist(), embeddings)

    # Save the store
    vector_store.save_local(store_path)
    print(f"FAISS store saved to {store_path}!")

# Example: Build and save FAISS store
build_and_save_faiss_store()


FAISS store saved to faiss_store!


In [16]:
from langchain.vectorstores import FAISS
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the FAISS store
def load_faiss_store(store_path="faiss_store"):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(store_path, embeddings,allow_dangerous_deserialization=True)
    return vector_store

# Initialize GPT-2 for generation
def setup_gpt2_pipeline():
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    return tokenizer, model

# Example: Load the store and GPT-2
vector_store = load_faiss_store()
tokenizer, gpt2_model = setup_gpt2_pipeline()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [19]:
import torch

def rag_with_gpt2(query, vector_store, tokenizer, gpt2_model):
    # Step 1: Retrieve relevant chunks
    retriever = vector_store.as_retriever()
    relevant_docs = retriever.get_relevant_documents(query)
    context = " ".join([doc.page_content for doc in relevant_docs])

    # Step 2: Prepare input for GPT-2
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=1024)

    # Set attention mask
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)


    outputs = gpt2_model.generate(
        inputs,
        attention_mask=attention_mask,
        max_new_tokens=200,
        num_beams=3,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

# Example: Query with RAG
query = "How to use print in python?"
response = rag_with_gpt2(query, vector_store, tokenizer, gpt2_model)
print("\n--- Generated Response ---")
print(response)


--- Generated Response ---
Context: In Python 3, print is a function, you need to call it like `print("hello world")`.
 In Python 3, `print` [became a function](http://docs.python.org/3.0/whatsnew/3.0.html#print-is-a-function). This means that you need to include parenthesis now like mentioned below:

```
print("Hello World")
```

 Unfortunately, there is no standard way to print using Python on all platforms. So you'll need to write your own wrapper function to print.

You need to [detect the OS](https://stackoverflow.com/questions/1854/python-what-os-am-i-running-on) your program is running on, then:

For Linux -

```
import subprocess
lpr =  subprocess.Popen("/usr/bin/lpr", stdin=subprocess.PIPE)
lpr.stdin.write(your_data_here)
```


For Windows: [http://timgolden.me.uk/python/win32_how_do_i/print.html](http://timgolden.me.uk/python/win32_how_do_i/print.html)

More resources:

[Print PDF document with python's win32print module?](https://stackoverflow.com/questions/1462842/print-pd